In [27]:
# Import the required libaries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [26]:
# Pull in the dataframe.
# Josh G
df_reviews = pd.read_csv('../../../../../Downloads/steam_reviews.csv', low_memory=False)
# Gabe
# df_reviews = pd.read_csv('steam_reviews.csv', low_memory=False)

In [ ]:
# Display the columns for the dataframe.
df_reviews.columns

In [ ]:
# Get and show a small sample.
user_rec_df = df_reviews[["app_id","author.steamid", "recommended"]].copy()
user_rec_df.sample(5)

In [ ]:
# going to be using thesee to create the user review matrix dimensions
n_ratings = len(user_rec_df)
n_games = len(user_rec_df['app_id'].unique())
n_users = len(user_rec_df['author.steamid'].unique())

print(n_ratings, n_games, n_users)

In [ ]:
# maps idices to users and game IDs
user_map = dict(zip(np.unique(user_rec_df["author.steamid"]), list(range(n_users))))
game_map = dict(zip(np.unique(user_rec_df["app_id"]), list(range(n_games))))

print(len(user_map) == n_users)
print(len(game_map) == n_games)

user_i_map = dict(zip(list(range(n_users)), np.unique(user_rec_df["author.steamid"])))
game_i_map = dict(zip(list(range(n_games)), np.unique(user_rec_df["app_id"])))

In [ ]:
# creating indices for csr_matrix
user_index = [user_map[i] for i in user_rec_df['author.steamid']]
game_index = [game_map[i] for i in user_rec_df['app_id']]

In [ ]:
# creates csr matrix
matrix = csr_matrix((user_rec_df["recommended"], (game_index, user_index)), shape=(n_games, n_users))

In [ ]:
# creating a dictionary to pull the name of games from based on IDs
game_names = dict(zip(df_reviews['app_id'], df_reviews['app_name']))

In [22]:
def find_related_games(game_id, k):
    # going to store the IDs for nearest games
    neighbour_ids = []
    
    # Honestly kinda confused as to what's going on here. I think the vectors are the euclidean distances 
    # of the other games in relation to the movie_id
    game_index = game_map[game_id]
    game_vector = matrix[game_index]
    k+=1
    # knn model fits created matrix so the 
    kNN = NearestNeighbors(algorithm = 'brute', metric='cosine')
    kNN.fit(matrix)
    game_vec = game_vector.reshape(1,-1)
    # uses kneighborst to fin the 'k' most related games
    distances, indices = kNN.kneighbors(game_vec, n_neighbors=k)
    # adds returned list of games to neighbor_ids
    for i in range(0,len(distances.flatten())):
        n = indices.flatten()[i]
        neighbour_ids.append(game_i_map[n])
    neighbour_ids.pop(0)
    
    # Print the data related to the game.
    print(f"Games related to: {game_names[game_id]}\n")
    for i in range(len(neighbour_ids)):
        print(game_names[neighbour_ids[i]])

In [ ]:
# Set a test game id and run the function.
game_id = 225540
 
find_related_games(game_id, k=10)

In [ ]:
# Perform another test to see the results.
game_id = 4000
  
find_related_games(game_id, k=10)

In [ ]:
# Perform another test to see the results.
game_id = 413150

find_related_games(game_id, k=10)